In [1]:
%pwd

'd:\\CarebotNew\\medical-chatbot\\research'

In [2]:
import os
desired_directory = r'd:\\CarebotNew\\medical-chatbot'
os.chdir(desired_directory)

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
import os
print(os.path.exists('Data/'))  # Check if the directory exists


True


In [5]:
#Extract the data from pdf file
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents            

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
#extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
#text_chunks

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

In [12]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

d:\CarebotNew\medical-chatbot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("length",len(query_result))

length 384


In [15]:
#query_result

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=os.environ.get('HUGGINGFACEHUB_API_TOKEN')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')


In [18]:
import os
import pinecone
from pinecone import ServerlessSpec

# Initialize Pinecone using the Pinecone class
pinecone_instance = pinecone.Pinecone(api_key=os.getenv('PINECONE_API_KEY'), environment="us-east-1")

# Specify your index name and other parameters
index_name = "carebot"

# Create the index using the Pinecone instance
pinecone_instance.create_index(
    name=index_name,
    dimension=384,  # Replace with your model dimensions
    metric="cosine",  # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


{
    "name": "carebot",
    "metric": "cosine",
    "host": "carebot-ns2kcsw.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
#pip install -U langchain-community
#pip install langchain-pinecone
#pip install transformers requests
#pip install groq langchain-groq

In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [25]:
#Load existing index

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
)

In [26]:
docsearch

In [1]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

NameError: name 'docsearch' is not defined

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

# 1. Load environment variables
load_dotenv()  # This loads the .env file

# 2. Initialize Groq - VERIFIED WORKING CONFIG
llm = ChatGroq(
    temperature=0.2,
    model_name="llama3-70b-8192",
    api_key=os.getenv("GROQ_API_KEY")  # This reads from .env
)

In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate


prompt_template = PromptTemplate.from_template("""
You are CareBot, a helpful, knowledgeable, and friendly AI healthcare assistant. You provide accurate, clear, and empathetic responses to user queries about medical conditions, symptoms, treatments, medications, and general health advice."

"Use the pieces of retrived context to answer "
"the question. If you don't know the answer, say that you "
"don't know. Use three sentences maximum and keep the" 
"Keep answers concise, trustworthy, and user-friendly."
"If the question is outside the medical domain, politely explain that your expertise is limited to healthcare."
"Your goal is to educate, support, and guide users—never replace real doctors."
"\n\n"

Context: {context}
Question: {input}
Answer:
""")


In [43]:
question_answer_chain = create_stuff_documents_chain(llm,prompt_template)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [1]:
response = rag_chain.invoke({"input": "what is a car?"})
print(response["answer"])

NameError: name 'rag_chain' is not defined